In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertModel, AutoModel
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torch
from torch.optim import AdamW
# 
from dataset import EssayDataset
from longDataset import LongEssayDataset
from bert_regression import BertRegressionModel
from hierarchicalBert import HierarchicalBert

c:\Users\User\Documents\Code\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Merge dataset

In [2]:
# df1 = pd.read_csv("dataset/analisis_essay.csv")
# df1['dataset'] = 'analisis_essay'
# df2 = pd.read_csv("dataset/asap.csv")
# df2['dataset'] = 'asap'
# df3 = pd.read_csv("dataset/cunlp.csv")
# df3['dataset'] = 'cunlp'
# df4 = pd.read_csv("dataset/sag.csv")
# df4['dataset'] = 'sag'
# df5 = pd.read_csv("dataset/sci.csv")
# df5['dataset'] = 'sci'
# df6 = pd.read_csv("dataset/stita.csv")
# df6['dataset'] = 'stita'

# # add null value to all dataframe that doesn't have question column
# for df in [df2, df3, df6]:
#     df['question'] = pd.NA

# df = pd.concat([df1, df2, df3, df4, df5, df6], ignore_index=True)
# df.shape

# df.to_csv("dataset/aes_dataset.csv", index=False)

### Preprocessing

In [12]:
df = pd.read_csv("dataset/aes_dataset.csv")
df.head()

,question,reference_answer,answer,score,dataset,max_length1,normalized_score
0,Jelaskan kegunaan karbohidrat untuk tubuh kita.,Fungsi karbohidrat adalah sebagai pemasok ener...,"sumber tenaga, pemanis alami, menjaga sistem i...",27.0,analisis_essay,65,0.27
1,Jelaskan kegunaan karbohidrat untuk tubuh kita.,Fungsi karbohidrat adalah sebagai pemasok ener...,"sebagai sumber energi, pemanis alami, menjaga ...",21.0,analisis_essay,66,0.21
2,Jelaskan kegunaan karbohidrat untuk tubuh kita.,Fungsi karbohidrat adalah sebagai pemasok ener...,1. Sebagai energi. 2. Sebagai memperlancaar pe...,42.0,analisis_essay,76,0.42
3,Jelaskan kegunaan karbohidrat untuk tubuh kita.,Fungsi karbohidrat adalah sebagai pemasok ener...,"untuk membuat kenyang, agar tidak lapar, agar ...",18.0,analisis_essay,67,0.18
4,Jelaskan kegunaan karbohidrat untuk tubuh kita.,Fungsi karbohidrat adalah sebagai pemasok ener...,Karbohidrat mempunyai peran penting untuk pros...,82.0,analisis_essay,105,0.82


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22406 entries, 0 to 22405
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   question          4859 non-null   object 
 1   reference_answer  22406 non-null  object 
 2   answer            22406 non-null  object 
 3   score             22406 non-null  float64
 4   dataset           22406 non-null  object 
 5   max_length1       22406 non-null  int64  
 6   normalized_score  22406 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 1.2+ MB


In [4]:
df['dataset'].value_counts()

dataset
asap              17043
sag                2558
analisis_essay     2162
stita               333
cunlp               171
sci                 139
Name: count, dtype: int64

In [5]:
# convert NaN to None
df['question'] = df['question'].replace({np.nan: None})

### Analysis Max Length of the Sub Token

In [6]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# dataset = EssayDataset(df, tokenizer, 512)

# df['max_length'] = df.index.map(dataset.get_max_length)
# len(df[(df['max_length'] > 510)])

In [7]:
tokenizer1 = BertTokenizer.from_pretrained("indobenchmark/indobert-lite-base-p2")
dataset = EssayDataset(df, tokenizer1, 512)

df['max_length1'] = df.index.map(dataset.get_max_length)
len(df[(df['dataset'] != 'analisis_essay')&(df['max_length1'] > 510)])

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


308

In [8]:
df['max_length1'].describe()

count    22406.000000
mean       161.063688
std        108.950303
min         30.000000
25%        114.000000
50%        150.000000
75%        186.000000
max       1832.000000
Name: max_length1, dtype: float64

## Normalize Score

In [9]:
def normalize_score(row, min_max_dict):
    min_score, max_score = min_max_dict[row['dataset']]
    return (row['score'] - min_score) / (max_score - min_score)

# Menghitung min dan max per kategori sebagai tuple
min_max_dict = df.groupby('dataset')['score'].agg(['min', 'max']).apply(tuple, axis=1).to_dict()

# Menambahkan kolom normalized_score
df['normalized_score'] = df.apply(lambda x: normalize_score(x, min_max_dict), axis=1)


In [10]:
df['normalized_score'].describe()

count    22406.000000
mean         0.384133
std          0.339251
min          0.000000
25%          0.000000
50%          0.333333
75%          0.666667
max          1.000000
Name: normalized_score, dtype: float64

In [11]:
# save clean dataframe
df.to_csv("dataset/aes_dataset.csv", index=False)

## Training

In [12]:
df_sag = df[df['dataset'] == 'sag']
df_sag.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2558 entries, 19376 to 21933
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   question          2558 non-null   object 
 1   reference_answer  2558 non-null   object 
 2   answer            2558 non-null   object 
 3   score             2558 non-null   float64
 4   dataset           2558 non-null   object 
 5   max_length1       2558 non-null   int64  
 6   normalized_score  2558 non-null   float64
dtypes: float64(2), int64(1), object(4)
memory usage: 159.9+ KB


In [13]:
df_test = df_sag[:12]
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12 entries, 19376 to 19387
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   question          12 non-null     object 
 1   reference_answer  12 non-null     object 
 2   answer            12 non-null     object 
 3   score             12 non-null     float64
 4   dataset           12 non-null     object 
 5   max_length1       12 non-null     int64  
 6   normalized_score  12 non-null     float64
dtypes: float64(2), int64(1), object(4)
memory usage: 768.0+ bytes


In [14]:
test = df[df['max_length1'] > 1000]

In [15]:
test_dataset = LongEssayDataset(test, tokenizer1, 512, 128)

In [126]:
dataloader = DataLoader(test_dataset, batch_size=4, collate_fn=lambda x: list(zip(*x)))

In [153]:
model = HierarchicalBert("indobenchmark/indobert-lite-base-p2")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.MSELoss()
model.train()

HierarchicalBert(
  (bert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertSdpaAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
 

In [154]:
for epoch in range(5):
    for batch, targets in dataloader:
        optimizer.zero_grad()
        # batch = batch.to(device)
        targets = torch.stack(targets).to(device)
        # forward pass
        predictions = model(batch).squeeze(1)

        print("Predictions:", predictions)
        print("Targets:", targets)
        # compute loss
        loss = criterion(predictions, targets)
        print("loss : ", loss.item())
        # backward pass
        loss.backward()
        optimizer.step()

len tensor :  1166
len tensor :  1018
Predictions: tensor([-0.1692, -0.1689], device='cuda:0', grad_fn=<SqueezeBackward1>)
Targets: tensor([0.8000, 0.8000], device='cuda:0')
loss :  0.9390792846679688
len tensor :  1065
len tensor :  1070
Predictions: tensor([0.1305, 0.1319], device='cuda:0', grad_fn=<SqueezeBackward1>)
Targets: tensor([1., 1.], device='cuda:0')
loss :  0.7548768520355225
len tensor :  1020
len tensor :  1025
Predictions: tensor([0.4216, 0.4258], device='cuda:0', grad_fn=<SqueezeBackward1>)
Targets: tensor([0.8000, 1.0000], device='cuda:0')
loss :  0.23644286394119263
len tensor :  1056
len tensor :  1123
Predictions: tensor([0.6767, 0.6784], device='cuda:0', grad_fn=<SqueezeBackward1>)
Targets: tensor([0.8000, 0.9100], device='cuda:0')
loss :  0.03441764786839485
len tensor :  1055
len tensor :  1045
Predictions: tensor([0.8852, 0.8847], device='cuda:0', grad_fn=<SqueezeBackward1>)
Targets: tensor([0.8000, 0.8000], device='cuda:0')
loss :  0.007216768339276314
len ten

KeyboardInterrupt: 